In [138]:
import yaml
import sagemaker
import boto3
import json

SETTING_FILE_PATH = "../config/settings.yaml"
DATA_FOLDER_PATH = "../avazu-ctr-prediction"

# AWS リソース設定
with open(SETTING_FILE_PATH) as file:
    aws_info = yaml.safe_load(file)
        
sess = sagemaker.Session()
role = aws_info['aws']['sagemaker']['role']
bucket = aws_info['aws']['sagemaker']['s3bucket']
region = aws_info['aws']['sagemaker']['region']

sm = boto3.client('sagemaker')
s3 = boto3.client('s3')


In [145]:
import os

import pandas as pd
from sklearn.model_selection import train_test_split

df_train = (
    pd.read_csv(os.path.join(DATA_FOLDER_PATH, "train_partial.csv"), dtype="object")
    .sort_values(by="hour")
    .reset_index(drop=True)
)

# WarmStartのための親・子データを作成
df_train_parent = df_train[: int(len(df_train) * 0.5)]
df_train_child = df_train[int(len(df_train) * 0.5) :]

# train, validation データに分割
df_train_parent, df_validation_parent = train_test_split(df_train_parent, train_size=0.8, random_state=0, shuffle=False)
df_train_child, df_validation_child = train_test_split(df_train_child, train_size=0.8, random_state=0, shuffle=False)


In [146]:
# S3にアップロード
prefix = 'sagemaker-hyperparameter-tuning'

train_parent_file = "train_parent.csv"
validation_parent_file = "validation_parent.csv"
train_child_file = "train_child.csv"
validation_child_file = "validation_child.csv"

df_train_parent.to_csv(train_parent_file, index=False)
df_validation_parent.to_csv(validation_parent_file, index=False)

df_train_child.to_csv(train_child_file, index=False)
df_validation_child.to_csv(validation_child_file, index=False)

s3_resource_bucket = boto3.Session().resource("s3").Bucket(bucket)

s3_resource_bucket.Object(os.path.join(prefix, "train_parent", train_parent_file)).upload_file(train_parent_file)
s3_resource_bucket.Object(os.path.join(prefix, "validation_parent", validation_parent_file)).upload_file(validation_parent_file)

s3_resource_bucket.Object(os.path.join(prefix, "train_child", train_child_file)).upload_file(train_child_file)
s3_resource_bucket.Object(os.path.join(prefix, "validation_child", validation_child_file)).upload_file(validation_child_file)


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [147]:
output_location = f"s3://{bucket}/{prefix}/output"

s3_train_parent_data = f"s3://{bucket}/{prefix}/train_parent/{train_parent_file}"
s3_validation_parent_data = f"s3://{bucket}/{prefix}/validation_parent/{validation_parent_file}"

s3_train_child_data = f"s3://{bucket}/{prefix}/train_child/{train_child_file}"
s3_validation_child_data = f"s3://{bucket}/{prefix}/validation_child/{validation_child_file}"

In [157]:
metrics_definitions = [
    {"Name": "train loss", "Regex": "train logloss: ([0-9\\.]+)"},
    {"Name": "train accuracy", "Regex": "train accuracy: ([0-9\\.]+)"},
    {"Name": "validation loss", "Regex": "validation logloss: ([0-9\\.]+)"},
    {"Name": "validation accuracy", "Regex": "validation accuracy: ([0-9\\.]+)"},
]

In [165]:
from sagemaker.tuner import ContinuousParameter
from sagemaker.tuner import CategoricalParameter

parent_hyperparameter_ranges = {
    "alpha": ContinuousParameter(0.00001, 0.00009, scaling_type="Linear"),
    "penalty": CategoricalParameter(['l1', 'l2']),
    "fit_intercept": CategoricalParameter([True, False]),
}

In [166]:
from time import gmtime, strftime

from sagemaker.sklearn.estimator import SKLearn

parent_train_job_name = "hpt-parent-training-job" + strftime("%Y%m%d%H%M", gmtime())

parent_estimator_parameters = {
    "entry_point": "trainer.py",
    "source_dir": "model",
    "framework_version": "0.23-1",
    "py_version": "py3",
    "instance_type": "ml.m5.large",
    "instance_count": 1,
    "output_path": output_location,
    "role": role,
    "base_job_name": parent_train_job_name,
}

parent_estimator = SKLearn(**parent_estimator_parameters)


INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.


In [167]:
from sagemaker.tuner import HyperparameterTuner

objective_metric_name = "validation loss"
base_parent_tuning_job_name = "sgd-classifier-parent"

tuner_parent = HyperparameterTuner(
    estimator=parent_estimator,
    base_tuning_job_name=base_parent_tuning_job_name,
    objective_type="Minimize",
    objective_metric_name=objective_metric_name,
    hyperparameter_ranges=parent_hyperparameter_ranges,
    metric_definitions=metrics_definitions,
    max_jobs=20,
    max_parallel_jobs=5,
    strategy="Bayesian",
    early_stopping_type="Auto",
)

In [168]:
parent_inputs = {"train": s3_train_parent_data, "validation": s3_validation_parent_data}
parent_job_name = "hpt-parent-" + strftime("%Y%m%d%H%M", gmtime())

tuner_parent.fit(
    inputs=parent_inputs,
    job_name=parent_job_name,
    wait=False,
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating hyperparameter tuning job with name: hpt-parent-202207091413


In [169]:
from sagemaker.analytics import HyperparameterTuningJobAnalytics

parent_tuning_job_name = tuner_parent.latest_tuning_job.job_name

parent_results = HyperparameterTuningJobAnalytics(sagemaker_session=sess, hyperparameter_tuning_job_name=parent_tuning_job_name)
df_parent = parent_results.dataframe()


In [173]:
df_parent.sort_values("FinalObjectiveValue", ascending=True)

,alpha,fit_intercept,penalty,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
12,0.000054,"""False""","""l2""",hpt-parent-202207091413-008-f12eb9c6,Completed,0.061261,2022-07-09 23:45:21+09:00,2022-07-09 23:47:28+09:00,127.0
10,0.000047,"""False""","""l2""",hpt-parent-202207091413-010-6a9bcee8,Completed,0.061583,2022-07-09 23:53:38+09:00,2022-07-09 23:55:55+09:00,137.0
6,0.000044,"""False""","""l2""",hpt-parent-202207091413-014-a4f41b63,Completed,0.061733,2022-07-10 00:10:42+09:00,2022-07-10 00:13:00+09:00,138.0
9,0.000062,"""False""","""l2""",hpt-parent-202207091413-011-7f80379d,Completed,0.061800,2022-07-09 23:57:47+09:00,2022-07-09 23:59:59+09:00,132.0
15,0.000038,"""False""","""l2""",hpt-parent-202207091413-005-a402187e,Completed,0.061953,2022-07-09 23:31:53+09:00,2022-07-09 23:34:00+09:00,127.0
4,0.000064,"""True""","""l2""",hpt-parent-202207091413-016-e393507d,Completed,0.061988,2022-07-10 00:19:23+09:00,2022-07-10 00:21:30+09:00,127.0
2,0.000052,"""False""","""l2""",hpt-parent-202207091413-018-90230654,Completed,0.061989,2022-07-10 00:27:38+09:00,2022-07-10 00:29:46+09:00,128.0
8,0.000057,"""False""","""l2""",hpt-parent-202207091413-012-45ba0cac,Completed,0.062106,2022-07-10 00:02:08+09:00,2022-07-10 00:04:26+09:00,138.0
1,0.000055,"""False""","""l2""",hpt-parent-202207091413-019-fd71733d,Completed,0.062203,2022-07-10 00:32:06+09:00,2022-07-10 00:34:18+09:00,132.0
5,0.000045,"""False""","""l2""",hpt-parent-202207091413-015-159b3620,Completed,0.062482,2022-07-10 00:15:02+09:00,2022-07-10 00:17:09+09:00,127.0


In [174]:
import bokeh
import bokeh.io

bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

import pandas as pd

df_parent_objective_value = df_parent[df_parent["FinalObjectiveValue"] > -float("inf")]

p = figure(
    plot_width=900,
    plot_height=400,
    x_axis_type="datetime",
    x_axis_label="datetime",
    y_axis_label=objective_metric_name,
)
p.circle(
    source=df_parent_objective_value, x="TrainingStartTime", y="FinalObjectiveValue", color="black"
)

show(p)

Loading BokehJS ...

## Warm Start

In [175]:
from sagemaker.tuner import WarmStartConfig
from sagemaker.tuner import WarmStartTypes

warm_start_config = WarmStartConfig(
    warm_start_type=WarmStartTypes.TRANSFER_LEARNING, parents={parent_tuning_job_name}
)

In [176]:
child_hyperparameter_ranges = {  
    "alpha": ContinuousParameter(0.00001, 0.00012, scaling_type="Linear"),
    "penalty": CategoricalParameter(['l1', 'l2']),
    "fit_intercept": CategoricalParameter([True, False]),
}

In [177]:
from time import gmtime, strftime

from sagemaker.sklearn.estimator import SKLearn

child_train_job_name = "hpt-child-training-job" + strftime("%Y%m%d%H%M", gmtime())

child_estimator_parameters = {
    "entry_point": "trainer.py",
    "source_dir": "model",
    "framework_version": "0.23-1",
    "py_version": "py3",
    "instance_type": "ml.m5.large",
    "instance_count": 1,
    "output_path": output_location,
    "role": role,
    "base_job_name": child_train_job_name,
}

child_estimator = SKLearn(**child_estimator_parameters)

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.


In [182]:
objective_metric_name = "validation loss"
base_child_tuning_job_name = "sgd-classifier-child"

tuner_child = HyperparameterTuner(
    estimator=child_estimator,
    base_tuning_job_name=base_child_tuning_job_name,
    objective_type="Minimize",
    objective_metric_name=objective_metric_name,
    hyperparameter_ranges=child_hyperparameter_ranges,
    metric_definitions=metrics_definitions,
    max_jobs=20,
    max_parallel_jobs=1,
    strategy="Bayesian",
    early_stopping_type="Auto",
    warm_start_config=warm_start_config,
)

In [183]:
child_inputs = {"train": s3_train_child_data, "validation": s3_validation_child_data}
child_job_name = "hpt-child-" + strftime("%Y%m%d%H%M", gmtime())

tuner_child.fit(
    inputs=child_inputs,
    job_name=child_job_name,
    wait=False,
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating hyperparameter tuning job with name: hpt-child-202207091544


In [184]:
from sagemaker.analytics import HyperparameterTuningJobAnalytics

child_results = HyperparameterTuningJobAnalytics(sagemaker_session=sess, hyperparameter_tuning_job_name=child_job_name)

df_child = child_results.dataframe()
df_child

,alpha,fit_intercept,penalty,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,0.000023,"""True""","""l2""",hpt-child-202207091544-020-8be1ae8c,Completed,0.017958,2022-07-10 02:08:24+09:00,2022-07-10 02:10:36+09:00,132.0
1,0.000026,"""True""","""l2""",hpt-child-202207091544-019-33ebd2e3,Completed,0.016434,2022-07-10 02:04:07+09:00,2022-07-10 02:06:24+09:00,137.0
2,0.000023,"""True""","""l2""",hpt-child-202207091544-018-f65eb508,Completed,0.016057,2022-07-10 01:59:44+09:00,2022-07-10 02:02:13+09:00,149.0
3,0.000024,"""True""","""l2""",hpt-child-202207091544-017-dca5acc7,Completed,0.015099,2022-07-10 01:55:26+09:00,2022-07-10 01:57:39+09:00,133.0
4,0.000024,"""False""","""l2""",hpt-child-202207091544-016-fe831895,Completed,0.016664,2022-07-10 01:51:24+09:00,2022-07-10 01:53:36+09:00,132.0
5,0.000021,"""True""","""l2""",hpt-child-202207091544-015-6f394554,Completed,0.016723,2022-07-10 01:46:58+09:00,2022-07-10 01:49:05+09:00,127.0
6,0.000024,"""False""","""l2""",hpt-child-202207091544-014-3f11c4e3,Completed,0.016822,2022-07-10 01:42:39+09:00,2022-07-10 01:44:57+09:00,138.0
7,0.000074,"""False""","""l2""",hpt-child-202207091544-013-f099a809,Completed,0.027178,2022-07-10 01:38:15+09:00,2022-07-10 01:40:28+09:00,133.0
8,0.000029,"""True""","""l2""",hpt-child-202207091544-012-5e6adb38,Completed,0.018244,2022-07-10 01:34:02+09:00,2022-07-10 01:36:15+09:00,133.0
9,0.000033,"""False""","""l2""",hpt-child-202207091544-011-cabb67e6,Completed,0.018977,2022-07-10 01:29:38+09:00,2022-07-10 01:31:55+09:00,137.0


In [185]:
import bokeh
import bokeh.io

bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

import pandas as pd

df_parent_objective_value = df_parent[df_parent["FinalObjectiveValue"] > -float("inf")]
df_child_objective_value = df_child[df_child["FinalObjectiveValue"] > -float("inf")]

p = figure(
    plot_width=900,
    plot_height=400,
    x_axis_type="datetime",
    x_axis_label="datetime",
    y_axis_label=objective_metric_name,
)
p.circle(
    source=df_parent_objective_value, x="TrainingStartTime", y="FinalObjectiveValue", color="black"
)
p.circle(
    source=df_child_objective_value,
    x="TrainingStartTime",
    y="FinalObjectiveValue",
    color="red",
)
show(p)

Loading BokehJS ...